In [1]:
from prody import *
from pylab import *
import json

In [2]:
pdb = {
    'cov2': parsePDB('6M0J').getHierView(),
    'cov' : parsePDB('2AJF').getHierView()
}

@> PDB file is found in working directory (6m0j.pdb).
@> 6558 atoms and 1 coordinate set(s) were parsed in 0.11s.
@> PDB file is found in working directory (2ajf.pdb).
@> 12777 atoms and 1 coordinate set(s) were parsed in 0.15s.


In [3]:
chains = {
    'cov' : {
        'spike': pdb['cov']['E'],
        'recep': pdb['cov']['A']
    },
    'cov2': {
        'spike': pdb['cov2']['E'],
        'recep': pdb['cov2']['A']
    }
}

# for finding interactions
threshold = 5

In [4]:
sequences = {
    'spikes' : [chains['cov']['spike'].getSequence(), chains['cov2']['spike'].getSequence()],
    'receps' : [chains['cov']['recep'].getSequence(), chains['cov2']['recep'].getSequence()]
}

In [5]:
alignments = {
    'spikes': buildMSA(sequences['spikes'], title='spike'),
    'receps': buildMSA(sequences['receps'], title='recep')
}

@> 2 sequence(s) with 204 residues were parsed in 0.00s.
@> 2 sequence(s) with 597 residues were parsed in 0.00s.


In [6]:
showAlignment(alignments['spikes'])

1              	---CPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLN----
2              	TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCF

1              	--VYADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYK
2              	TNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYL

1              	YRYLRHGKLRPFERDISNVPFSPDGKPCT-PPALNCYWPLNDYGFYTTTGIGYQPYRVVV
2              	YRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVV

1              	LSFE--------------------
2              	LSFELLHAPATVCG----------



In [7]:
residues = {
    'cov': {
        'spike': [],
        'recep': []
    },
    'cov2': {
        'spike': [],
        'recep': []
    }
}

for complex in ['cov', 'cov2']:
    for chain in ['spike', 'recep']:
        for resnum in sorted([int(res) for res in set(chains[complex][chain].getResnums())]):
            residue = chains[complex][chain].getResidue(int(resnum))
            if residue.getResname() not in ['HOH', 'NAG']:
                residues[complex][chain].append(residue)


In [8]:
aligned_sequences = {
    'cov': {
        'spike': str(alignments['spikes'][0]),
        'recep': str(alignments['receps'][0])
     },
     'cov2': {
         'spike': str(alignments['spikes'][1]),
         'recep': str(alignments['receps'][1])
     }
}

In [9]:
# alignment map of all residues
maps = {
    'spike': [],
    'recep': []
}

complex1 = 'cov'
complex2 = 'cov2'
for chain in ['spike', 'recep']:
        
    s1 = 0 # residue index for cov  spike residues
    s2 = 0 # residue index for cov2 spike residues

    for i in range(len(aligned_sequences[complex1][chain])):
        # get character from aligned sequences
        char1 = aligned_sequences[complex1][chain][i]
        char2 = aligned_sequences[complex2][chain][i]

        if char1 != '-' and char2 != ' ':
            resnum1 = residues[complex1][chain][s1].getResnum()
            resnum2 = residues[complex2][chain][s2].getResnum() 
            maps[chain].append((resnum1, resnum2))

        if char1 != '-':
            s1 += 1
        if char2 != '-':
            s2 += 1

        if s1 == len(residues[complex1][chain]) or s2 == len(residues[complex2][chain]):
           break    




In [10]:
print(len(residues['cov2']['spike']))
print(len(aligned_sequences['cov']['spike']))


194
204


In [11]:
showAlignment(alignments['spikes'])

1              	---CPFGEVFNATKFPSVYAWERKKISNCVADYSVLYNSTFFSTFKCYGVSATKLN----
2              	TNLCPFGEVFNATRFASVYAWNRKRISNCVADYSVLYNSASFSTFKCYGVSPTKLNDLCF

1              	--VYADSFVVKGDDVRQIAPGQTGVIADYNYKLPDDFMGCVLAWNTRNIDATSTGNYNYK
2              	TNVYADSFVIRGDEVRQIAPGQTGKIADYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYL

1              	YRYLRHGKLRPFERDISNVPFSPDGKPCT-PPALNCYWPLNDYGFYTTTGIGYQPYRVVV
2              	YRLFRKSNLKPFERDISTEIYQAGSTPCNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVV

1              	LSFE--------------------
2              	LSFELLHAPATVCG----------



In [12]:
interactions = {
    'cov' : findNeighbors(atoms=chains['cov' ]['spike'], radius=threshold, atoms2=chains['cov' ]['recep']),
    'cov2': findNeighbors(atoms=chains['cov2']['spike'], radius=threshold, atoms2=chains['cov2']['recep'])
}

In [13]:
spike_residues = {}
ace2_residues = {}

residue_links = {} 

In [14]:
# function to get residue data
def getResidueData(residue, complex):
    # getting a Residue involves making a Hierarchial View
    # which is a lot of computation
    # so it's easier to get the data from any one atom

    data = {}

    data['pdb']     = str(residue.getChain().getAtomGroup())[-4:]
    data['id']      = complex + '_' + str(residue.getResindex())
    data['chain']   = residue.getChid()
    data['resnum']  = int(residue.getResnum())
    data['aa']      = residue.getResname()
    data['name']    = data['chain'] + str(data['resnum']) + '_' + data['aa']
    data['bonds']   = 1 # number of atoms this residue's atoms are bonded with`
    
    return data

In [15]:
# store interacting nodes and links 

nodes = {
    'cov' : {
        'spike': {},
        'recep': {}
    },
    'cov2': {
        'spike': {},
        'recep': {}
    }
}

links = {
    'cov' : {},
    'cov2': {}
}

for complex in ['cov', 'cov2']:

    for pair in interactions[complex]:
        atom1, atom2, distance = pair

        atom1_resindex = atom1.getResindex()
        atom2_resindex = atom2.getResindex()
        atom1_complex_resindex = complex + '_' + str(atom1_resindex)
        atom2_complex_resindex = complex + '_' + str(atom2_resindex)
        
        # add if residue is not present
        if atom1_complex_resindex not in nodes[complex]['spike'].keys():
            # can't get residue from atom
            # get residue number from atom
            # select residue by residue number from chain
            # obtain data using custom function
            atom1_resnum  = atom1.getResnum()
            residue1      = chains[complex]['spike'].getResidue(atom1_resnum)
            residue1_data = getResidueData(residue1, complex)
            nodes[complex]['spike'][atom1_complex_resindex] = residue1_data
        else:
            # increment number of connections for atom and residue
            print("hello")
            nodes[complex]['spike'][atom1_complex_resindex]['bonds'] += 1

        if atom2_complex_resindex not in nodes[complex]['recep'].keys():
            atom2_resnum  = atom2.getResnum()
            residue2      = chains[complex]['recep'].getResidue(atom2_resnum)
            residue2_data = getResidueData(residue2, complex)
            nodes[complex]['recep'][atom2_complex_resindex] = residue2_data
        else:
            # increment number of connections for atom and residue
            print("hi")
            nodes[complex]['recep'][atom2_complex_resindex]['bonds'] += 1

        if (atom1_complex_resindex, atom2_complex_resindex) in links[complex].keys():
            links[complex][(atom1_complex_resindex, atom2_complex_resindex)]['strength'] += 1
        else:
            links[complex][(atom1_complex_resindex, atom2_complex_resindex)] = {
                "source":   atom1_complex_resindex,
                "target":   atom2_complex_resindex,
                "strength": 1
            }

In [16]:
# extract resnum of interacting residues 

interacting_residues = {
    'cov':  {
        'spike': [],
        'recep': []
    },
    'cov2': {
        'spike': [],
        'recep': []
    }
}

for complex in ['cov', 'cov2']:
    for chain in ['spike', 'recep']:
        for resindex in nodes[complex][chain].keys():
            resnum = nodes[complex][chain][resindex]['resnum'] 
            interacting_residues[complex][chain].append((resindex, resnum))


In [17]:
complex1 = 'cov'
complex2 = 'cov2'

for chain in ['spike', 'recep']:
    counter = 0
    for a_index, a_num in interacting_residues[complex1][chain]:
        for b_index, b_num in interacting_residues[complex2][chain]:
            if (a_num, b_num) in maps[chain]:
                print('match!')
                nodes[complex1][chain][a_index]['corresp'] = chain + '_' + str(counter)
                nodes[complex2][chain][b_index]['corresp'] = chain + '_' + str(counter)
                counter += 1

match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!
match!


In [18]:
nodes_json = []
dictionaries = [nodes['cov']['spike'], nodes['cov']['recep'], nodes['cov2']['spike'], nodes['cov2']['recep']]
for dictionary in dictionaries:
    for key in dictionary.keys():
        nodes_json.append(dictionary[key])

links_json = []
dictionaries = [links['cov'], links['cov2']]
for dictionary in dictionaries:
    for key in dictionary.keys():
        links_json.append(dictionary[key])


interactions_json = {
    "nodes": nodes_json,
    "links": links_json
}

with open('interactions.json', 'w') as file:
    json.dump(interactions_json, file, indent=2)